In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [30]:
df = pd.read_csv("/content/piper_robot_arm_end_pose_and_joint_angle_data_more_data.csv")
df = df.iloc[1:,]
df.drop("Unnamed: 0",axis=1,inplace=True)
df.head()


,joint1,joint2,joint3,joint4,joint5,joint6,X_axis,Y_axis,Z_axis
1,0,0,0,0,0,0,56127,0,213266
2,0,0,0,0,0,0,56127,0,213266
3,-28,0,-27,0,-692,0,55999,-28,214521
4,-28,0,37,0,547,0,56232,-28,212180
5,0,0,0,0,0,0,56127,0,213266


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

In [33]:
# 1. Spherical Coordinates (and Distance from Base)
# Assuming X_axis, Y_axis, Z_axis are Cartesian coordinates
# Calculate radius (distance from base/origin)
df['distance_from_base'] = np.sqrt(df['X_axis']**2 + df['Y_axis']**2 + df['Z_axis']**2)

In [34]:
df.head()

,joint1,joint2,joint3,joint4,joint5,joint6,X_axis,Y_axis,Z_axis,distance_from_base
1,0,0,0,0,0,0,56127,0,213266,220528.063713
2,0,0,0,0,0,0,56127,0,213266,220528.063713
3,-28,0,-27,0,-692,0,55999,-28,214521,221709.603369
4,-28,0,37,0,547,0,56232,-28,212180,219504.876957
5,0,0,0,0,0,0,56127,0,213266,220528.063713


In [35]:
# Calculate azimuth (phi) - angle in XY plane
# Using arctan2 for correct quadrant handling
df['azimuth'] = np.arctan2(df['Y_axis'], df['X_axis'])

# Calculate elevation (theta) - angle from XY plane
# Avoid division by zero for points exactly on the XY plane
df['elevation'] = np.arctan2(df['Z_axis'], np.sqrt(df['X_axis']**2 + df['Y_axis']**2))
df.head()

,joint1,joint2,joint3,joint4,joint5,joint6,X_axis,Y_axis,Z_axis,distance_from_base,azimuth,elevation
1,0,0,0,0,0,0,56127,0,213266,220528.063713,0.000000,1.313453
2,0,0,0,0,0,0,56127,0,213266,220528.063713,0.000000,1.313453
3,-28,0,-27,0,-692,0,55999,-28,214521,221709.603369,-0.000500,1.315452
4,-28,0,37,0,547,0,56232,-28,212180,219504.876957,-0.000498,1.311732
5,0,0,0,0,0,0,56127,0,213266,220528.063713,0.000000,1.313453


In [36]:
# 2. Polynomial Features / Combinations of Axes
# Create polynomial features up to degree 2 for X_axis, Y_axis, Z_axis
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[['X_axis', 'Y_axis', 'Z_axis']])
poly_feature_names = poly.get_feature_names_out(['X_axis', 'Y_axis', 'Z_axis'])


In [37]:
# Create a DataFrame for polynomial features
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names, index=df.index)


In [38]:
# Combine all features (original X,Y,Z, new spherical, and polynomial)
# Drop original X,Y,Z from poly_df to avoid duplication if they are already in main df
# Or, simply use selected columns for features
X_engineered = pd.concat([df[['X_axis', 'Y_axis', 'Z_axis', 'distance_from_base', 'azimuth', 'elevation']], poly_df], axis=1)


In [40]:
X_engineered.head()

,X_axis,Y_axis,Z_axis,distance_from_base,azimuth,elevation,X_axis,Y_axis,Z_axis,X_axis^2,X_axis Y_axis,X_axis Z_axis,Y_axis^2,Y_axis Z_axis,Z_axis^2
1,56127,0,213266,220528.063713,0.000000,1.313453,56127.0,0.0,213266.0,3.150240e+09,0.0,1.196998e+10,0.0,0.0,4.548239e+10
2,56127,0,213266,220528.063713,0.000000,1.313453,56127.0,0.0,213266.0,3.150240e+09,0.0,1.196998e+10,0.0,0.0,4.548239e+10
3,55999,-28,214521,221709.603369,-0.000500,1.315452,55999.0,-28.0,214521.0,3.135888e+09,-1567972.0,1.201296e+10,784.0,-6006588.0,4.601926e+10
4,56232,-28,212180,219504.876957,-0.000498,1.311732,56232.0,-28.0,212180.0,3.162038e+09,-1574496.0,1.193131e+10,784.0,-5941040.0,4.502035e+10
5,56127,0,213266,220528.063713,0.000000,1.313453,56127.0,0.0,213266.0,3.150240e+09,0.0,1.196998e+10,0.0,0.0,4.548239e+10


In [41]:
# Ensure no duplicate columns
X_engineered = X_engineered.loc[:,~X_engineered.columns.duplicated()].copy()

In [42]:
# Define target (y) - joint angles remain the same
y = df[['joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6']]


In [43]:
# Split the data into training and testing sets
X_train_eng, X_test_eng, y_train, y_test = train_test_split(X_engineered, y, test_size=0.2, random_state=42)


In [44]:
# Train the RandomForest Regressor model with engineered features
rf_model_engineered = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_engineered.fit(X_train_eng, y_train)

RandomForestRegressor(random_state=42)

In [46]:
# Make predictions on the test set
y_pred_engineered = rf_model_engineered.predict(X_test_eng)

# Evaluate the model using R2 score for each joint
r2_scores_engineered = []
for i, joint in enumerate(y.columns):
    score = r2_score(y_test[joint], y_pred_engineered[:, i])
    r2_scores_engineered.append(score)
    print(f"R2 score for {joint} with engineered features: {score:.4f}")

R2 score for joint1 with engineered features: 0.7556
R2 score for joint2 with engineered features: 0.9420
R2 score for joint3 with engineered features: 0.9525
R2 score for joint4 with engineered features: 0.3575
R2 score for joint5 with engineered features: 0.4195
R2 score for joint6 with engineered features: 0.4775


In [47]:
# Calculate and print the average R2 score
average_r2_score_engineered = sum(r2_scores_engineered) / len(r2_scores_engineered)
print(f"\nAverage R2 score across all joints with engineered features: {average_r2_score_engineered:.4f}")



Average R2 score across all joints with engineered features: 0.6507


In [51]:
X_test_eng.head()

,X_axis,Y_axis,Z_axis,distance_from_base,azimuth,elevation,X_axis^2,X_axis Y_axis,X_axis Z_axis,Y_axis^2,Y_axis Z_axis,Z_axis^2
781,471654,-214724,186810,550873.731441,-0.427217,0.345977,2.224575e+11,-1.012754e+11,8.810968e+10,4.610640e+10,-4.011259e+10,3.489798e+10
582,-103281,-71100,229971,261932.884155,-2.538685,1.071619,1.066696e+10,7.343279e+09,-2.375163e+10,5.055210e+09,-1.635094e+10,5.288666e+10
1753,-393306,135023,430889,598820.632983,2.810897,0.803172,1.546896e+11,-5.310536e+10,-1.694712e+11,1.823121e+10,5.817993e+10,1.856653e+11
1314,43119,-291483,142301,327217.300965,-1.423932,0.449908,1.859248e+09,-1.256846e+10,6.135877e+09,8.496234e+10,-4.147832e+10,2.024957e+10
1560,-17664,-2983,354408,354860.459405,-2.974297,1.520293,3.120169e+08,5.269171e+07,-6.260263e+09,8.898289e+06,-1.057199e+09,1.256050e+11


In [61]:
def predict_joint_angles(X,Y,Z):
  df = pd.DataFrame({'X_axis': [X], 'Y_axis': [Y], 'Z_axis': [Z]})
  df['distance_from_base'] = np.sqrt(df['X_axis']**2 + df['Y_axis']**2 + df['Z_axis']**2)
  df['azimuth'] = np.arctan2(df['Y_axis'], df['X_axis'])
  df['elevation'] = np.arctan2(df['Z_axis'], np.sqrt(df['X_axis']**2 + df['Y_axis']**2))

  poly = PolynomialFeatures(degree=2, include_bias=False)
  poly_features = poly.fit_transform(df[['X_axis', 'Y_axis', 'Z_axis']])
  poly_feature_names = poly.get_feature_names_out(['X_axis', 'Y_axis', 'Z_axis'])
  poly_df = pd.DataFrame(poly_features, columns=poly_feature_names, index=df.index)
  X_engineered = pd.concat([df[['X_axis', 'Y_axis', 'Z_axis', 'distance_from_base', 'azimuth', 'elevation']],
                            poly_df[["X_axis^2","X_axis Y_axis","X_axis Z_axis","Y_axis^2","Y_axis Z_axis","Z_axis^2"]]], axis=1)
  y_pred_engineered = rf_model_engineered.predict(X_engineered)
  print(y_pred_engineered[0])
predict_joint_angles(3,4,5)

[  1265.07  80992.63 -11124.59  13786.17 -37224.24  92656.11]
